In [ ]:
import numpy as np
import cupy as cp

import util
import NN0 as NN

from sklearn.metrics import accuracy_score

from datetime import datetime 


def Eval(ts_y,ts_x):
    def predict(X):
        total = cp.zeros((X.shape[0],10),dtype=cp.float32)
        for i in range(X.shape[0] // batch_size):
            X0 = X[i * batch_size : (i + 1) * batch_size]
            for Layer in sequential:
                X0 = Layer.Forward(X0)
            total[i * batch_size : (i + 1) * batch_size] = X0
        return total
    pd = predict(ts_x)
    pdinx = pd.get().argmax(axis=1)
    return "\tAccuracy_score: " + str(accuracy_score(ts_y, pdinx))

In [ ]:
tr_x,tr_y,ts_x,ts_y= util.load_SVHN()

In [ ]:
batch_size = 10
 
# loss = NN.MSEloss(10)
loss = NN.CrossEntropyloss()

lr = 0.01

In [ ]:
sequential = [
    #### Conv layers:
    NN.Conv((batch_size,3,32,32),(64,3,3,3),stride=1,padding=1),
    NN.Relu(),
    NN.MaxPool((batch_size, 64, 32, 32) ,size = 2, stride = 2),
    NN.Conv((batch_size,64,16,16),(128,64,3,3),stride=1,padding=1),
    NN.Relu(),
    NN.Conv((batch_size,128,16,16),(256,128,3,3),stride=1,padding=1),
    NN.Relu(),
    NN.MaxPool((batch_size, 256, 16, 16) ,size = 2, stride = 2),
    NN.Conv((batch_size,256,8,8),(512,256,3,3),stride=1,padding=1),
    NN.Relu(),
    NN.AvgPool((batch_size, 512, 8, 8) ,size = 2, stride = 2),
    ######   FC Layers:
    NN.Reshape((batch_size, 512 , 4 , 4) , (batch_size, 512*4*4)),
    NN.Linear(batch_size,512*4*4, 512),
    NN.Relu(),
    NN.Linear(batch_size,512, 512),
    NN.Relu(),
    NN.Linear(batch_size,512, 10)
]

In [ ]:
start_time = datetime.now() 
print("[INFO]  Train started!\t", start_time)

for ep in range(100):
    for i in range(tr_x.shape[0] // batch_size):
        X = tr_x[i * batch_size : (i + 1) * batch_size]
        Y = tr_y[i * batch_size : (i + 1) * batch_size]
        
        for Layer in sequential:
            X = Layer.Forward(X)

        dY = loss.Backward(X, Y)

        for Layer in reversed(sequential):
            dY = Layer.Backward(dY)

        for Layer in sequential:
            Layer.Update(lr=lr)

    if ep % 1 == 0:
        print(
            f"[{ep}] \t--> {Eval(ts_y,ts_x)}  \t\t/ loss mini-batch : {loss.LossVal:.6f}"
        )

time_elapsed = (datetime.now() - start_time )
print("[INFO]  Train finished!\t", datetime.now() , "\t Run time: ",time_elapsed)